# Fact Checking
A simple example on how to use the vector database for a fact-checking system with Wikidata statements

In [ ]:
import requests

HEADERS = {
    'User-Agent': 'Fact-Checker/1.0 (embeddings@wikimedia.de)'
}
LANG = 'en'
INCLUDE_EXTERNAL_IDS = False

# Define the claim to be checked
claim = 'Albert Einstein was a theoretical physicist who developed the theory of relativity.'

### Get from the vector database the Wikidata items and properties that are relevant to the query

In [ ]:
# Get relevant Wikidata items
items = requests.get(
    'https://wd-vectordb.wmcloud.org/item/query',
    params={'query': claim, 'lang': LANG},
    headers=HEADERS,
)
items = items.json()

# Get relevant Wikidata properties
properties = requests.get(
    'https://wd-vectordb.wmcloud.org/property/query',
    params={'query': claim, 'lang': LANG},
    headers=HEADERS,
)
properties = properties.json()

### Get all statements of each item

In [ ]:
def get_statements(qid):
    params = {
        'id': qid,
        'external_ids': INCLUDE_EXTERNAL_IDS,
        'format': 'json'
    }

    url = "https://wd-textify.toolforge.org"
    results = requests.get(url, params=params, headers=HEADERS)
    results.raise_for_status()

    text = results.json()
    return text

for i in range(len(items)):
    item_info = get_statements(items[i]['QID'])
    items[i]['label'] = item_info['label']
    items[i]['claims'] = item_info['claims']

### Sort statements by vector similarity

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-0.6B-FP8"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

# prepare the model input
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/894 [00:00<?, ?B/s]

ValueError: The checkpoint you are trying to load has model type `qwen3` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.